# Supplementary material

In [ ]:
%reset -f

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dadapy import Data

plt.rcParams['text.usetex'] = True

In [ ]:
NUM_COLORS1 = 33
cm = plt.get_cmap('Wistia')
colors1 = [cm(1.*i/NUM_COLORS1) for i in range(NUM_COLORS1)]
colors1_test = np.array(colors1)
colors1_test = colors1_test[::-1]

NUM_COLORS2 = 33 * 2
colors2 = [cm(1.*i/NUM_COLORS2) for i in range(NUM_COLORS2)]

NUM_COLORS_g = 10
cm2 = plt.get_cmap('Set3')
colors_g = [cm2(1.*i/NUM_COLORS_g) for i in range(NUM_COLORS_g)]

pd.set_option('display.precision',12)
np.set_printoptions(precision=12)

In [ ]:
# Import the datasets
files = ["../GPNowcastingForecasting/daily_data.txt"]
file_labels = ["day"]

X_labels = np.array(['GPR', 'VSTOXX', 'Unc. EUR/USD', 'Unc. EUR/JPY', 'Unc. EUR/GBP', 'Unc. EUR/CHF',
       'Nat. Gas', 'Elec. p. SPAIN', 'Elec. p. GER', 'Elec. p. IT',
       'Elec. p. FR', 'Brent Oil', 'Coal', 'Gold', 'EUR/USD Spot',
       'EUR/JPY Spot', 'EUR/GBP Spot', 'EUR/CHF Spot',
       'Bloom. Energy p. ind.', 'Sol. ESG fossil ind.',
       'SP Euro50 Env. ind.', 'MSCI Ener. sec. ind.', 'ERIX ind.', 'EUROnext100', 'IBEX35', 'DAX',
       'CAC', 'FTSEmib', 'EUROSTOXX Elect. ind.', '10 Year Bond y.', '3 Month Bond y.',
       'Inflation', 'GDP'])

In [ ]:
data_sets = []
for file in files:
    data = np.genfromtxt(file)
    data_sets.append(data)

data_daily = np.genfromtxt(files[0])

# Define the informative pool and target set
target_d = []
X = []

target_d = data_daily[:, 0][:, None]
X = data_daily[:, 1:]

# Convert the X as a dataframe with column names equal to the X_labels
X = pd.DataFrame(X, columns=X_labels)

X_labels = X.columns

# Define the target and the informative pool as numpy arrays
X = X.to_numpy(dtype=np.float64)[:,]

d_t = Data(target_d, maxk = target_d.shape[0]-1)
d_t.compute_distances()

d_X = Data(X, maxk = X.shape[0]-1 )
d_X.compute_distances()

sing_imbalances = d_X.return_inf_imb_target_all_coords(target_ranks = d_t.dist_indices, k = 5)

sort_idx = np.argsort(sing_imbalances[1])